### Chapter 5 Select Applied Solutions

In [2]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize)
from sklearn.model_selection import train_test_split
from functools import partial
from sklearn.base import clone
import pandas as pd



### Problem 5 

In [32]:
Default = load_data("Default")
Default.shape

(10000, 4)

In [53]:
Error = []
for i in range (1,4):
    rng = np.random.default_rng()
    value = rng.random()
    Auto_train, Auto_valid = train_test_split(Default, test_size = value, random_state = 0 ) 
    X_train, Y_train = Auto_train[["income", 'balance']], Auto_train['default']
    X_valid, Y_valid = Auto_valid[["income", 'balance']], Auto_valid['default']

    X_train = sm.add_constant(X_train)
    X_valid = sm.add_constant(X_valid)

    glm_train = sm.GLM(Y_train == "Yes",
                    X_train,
                    family=sm.families.Binomial())
    results = glm_train.fit()
    probs = results.predict(exog=X_valid)

    preds = np.where(probs >0.5, "Yes", "No")
    Error.append([np.mean(preds != Y_valid), value]) 

Error


[[0.028795811518324606, 0.4965520405008923],
 [0.027371578552680915, 0.8000775260329769],
 [0.027654540076900325, 0.6761750967991185]]

Very similar results across the board even with different percentages of the data that were included in the training set 

In [79]:
Error = []
for i in range (1,4):
    rng = np.random.default_rng()
    value = rng.random()
    Auto_train, Auto_valid = train_test_split(Default, test_size = value, random_state = 0 ) 
    X_train, Y_train = Auto_train[["income", 'balance', 'student']], Auto_train['default']
    X_valid, Y_valid = Auto_valid[["income", 'balance', 'student']], Auto_valid['default']

     # Convert categorical variable to numeric dummy
    X_train = pd.get_dummies(X_train, drop_first=True)
    X_valid = pd.get_dummies(X_valid, drop_first=True)

    # Add constant for intercept (important in statsmodels)
    X_train = sm.add_constant(X_train)
    X_valid = sm.add_constant(X_valid)

    X_train = X_train.astype(float)
    X_valid = X_valid.astype(float)
    
    glm_train = sm.GLM(Y_train == "Yes",
                    X_train,
                    family=sm.families.Binomial())
    results = glm_train.fit()
    probs = results.predict(exog=X_valid)

    preds = np.where(probs >0.5, "Yes", "No")
    Error.append([np.mean(preds != Y_valid), value]) 

Error

[[0.028821410190427176, 0.19420584237465222],
 [0.02791023842917251, 0.7129672606353645],
 [0.02938871473354232, 0.5103738705710762]]

In [83]:
Error = []
model_spec = MS(['income', 'balance', 'student'])

for i in range(1, 4):
    rng = np.random.default_rng()
    value = rng.random()

    Auto_train, Auto_valid = train_test_split(Default, test_size=value, random_state=0)

    Y_train = (Auto_train['default'] == "Yes").astype(int)
    Y_valid = Auto_valid['default']

    # Apply model spec to build the design matrix
    X_train = clone(model_spec).fit_transform(Auto_train)
    X_valid = clone(model_spec).fit_transform(Auto_valid)

    glm_train = sm.GLM(Y_train, X_train, family=sm.families.Binomial())
    results = glm_train.fit()

    probs = results.predict(X_valid)
    preds = np.where(probs > 0.5, "Yes", "No")

    Error.append([np.mean(preds != Y_valid), value])

Error


[[0.02834549878345499, 0.8219901135169876],
 [0.028994742711486377, 0.627657751190813],
 [0.02716868114236817, 0.9348005127967696]]

Slight increase in the overall prediction error rate when including student in the predictions

### Problem 6

In [73]:
X_train, Y_train = Default[["income", 'balance']], Default['default']

X_train = sm.add_constant(X_train)
X_valid = sm.add_constant(X_valid)

glm_train = sm.GLM(Y_train == "Yes",
                    X_train,
                    family=sm.families.Binomial())
results = glm_train.fit()

summarize(results)



,coef,std err,z,P>|z|
const,-11.540500,0.435000,-26.544,0.0
income,0.000021,0.000005,4.174,0.0
balance,0.005600,0.000000,24.835,0.0


In [74]:
def boot_fn(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    results = sm.GLM(Y_ == "Yes",
                    X_,
                    family=sm.families.Binomial()).fit()
    return results.params


def boot_SE(func,
            D,
            n=None,
            B=1000,
            seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index,
                         n,
                         replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)

In [75]:
pred_function = partial(boot_fn, MS(["income", "balance"]), "default")

In [76]:
rng = np.random.default_rng(0)
np.array([pred_function(Default,
          rng.choice(Default.index,
                     Default.shape[0],
                     replace=True)) for _ in range(10)])

array([[-1.16416373e+01,  1.87775777e-05,  5.73877605e-03],
       [-1.27619965e+01,  3.20594655e-05,  6.16200434e-03],
       [-1.12850364e+01,  1.59221870e-05,  5.61832222e-03],
       [-1.09975828e+01,  1.40723398e-05,  5.41168597e-03],
       [-1.13173469e+01,  1.12728778e-05,  5.70216361e-03],
       [-1.17516107e+01,  1.85974460e-05,  5.83443562e-03],
       [-1.12884834e+01,  1.52822182e-05,  5.53172383e-03],
       [-1.13883312e+01,  1.73720495e-05,  5.70192972e-03],
       [-1.11098351e+01,  2.33921172e-05,  5.28010522e-03],
       [-1.10505563e+01,  1.50937413e-05,  5.46083916e-03]])

In [ ]:
pred_se = boot_SE(pred_function,
                Default,
                B=1000,
                seed=10)
pred_se

intercept    0.425280
income       0.000005
balance      0.000227
dtype: float64

rouhgly similar resutls though we knw the bootstrap method will produce more indicative results as the MSE does not assume the model is correct

### Problem 9 

In [85]:
Boston  = load_data("Boston")
mean_est = Boston["medv"].mean()
mean_est 

22.532806324110677

In [96]:
se_est = Boston["medv"].std()/np.sqrt(Boston.shape[0])
se_est

0.40886114749753505

In [101]:
def SE_est(D, idx):
    return D["medv"].iloc[idx].mean()

boot_se = boot_SE(SE_est, Boston, n=Boston.shape[0], B=1000, seed=0)
print("Bootstrap SE:", boot_se)



Bootstrap SE: 0.41253476750888246


Comparable results 

In [104]:
low_95CI = mean_est - 2*boot_se
high_95CI = mean_est + 2*boot_se
print(f"Lower bounds: {low_95CI}, Upper bounds: {high_95CI}")


Lower bounds: 21.707736789092912, Upper bounds: 23.35787585912844


In [107]:
median_est = Boston["medv"].median()
median_est

21.2

In [106]:
def MED_est(D,idx):
    return D["medv"].iloc[idx].median()

boot_med = boot_SE(MED_est, Boston, n=Boston.shape[0], B=1000, seed=0)
print("Bootstrap Median SE est:", boot_med)

Bootstrap Median SE est: 0.36944622071302163


In [109]:
tenth_est = np.percentile(Boston["medv"], 10)
tenth_est

12.75

In [112]:
def Ten_est(D,idx):
    return D["medv"].iloc[idx].quantile(0.1)

boot_ten = boot_SE(Ten_est, Boston, n =Boston.shape[0], B = 1000, seed =0)
print("Bootstrap Tenth percentile SE est:", boot_ten)

Bootstrap Tenth percentile SE est: 0.5034541091295026
